# 2 Pandas - Data Cleaning

Our dataframe is made out of 700k+ rows, however we only have salary information for approx. 33k rows (the remainder of rows are NaN).

We have a coworker that wants to have access to a dataset where all the NaN salary values are replaced by the median values for a similar role. They also want any duplicate kob postings to be deleted.

In [31]:
# importing libraries
import pandas as pd
from datasets import load_dataset
import matplotlib.pyplot as plt

# Loading data
dataset = load_dataset("lukebarousse/data_jobs")
df = dataset["train"].to_pandas()

# Data cleanup - update date column
df["job_posted_date"] = pd.to_datetime(df["job_posted_date"])

In [32]:
df

,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills,job_type_skills
0,Senior Data Engineer,Senior Clinical Data Engineer / Principal Clin...,"Watertown, CT",via Work Nearby,Full-time,False,"Texas, United States",2023-06-16 13:44:15,False,False,United States,None,NaN,NaN,Boehringer Ingelheim,None,None
1,Data Analyst,Data Analyst,"Guadalajara, Jalisco, Mexico",via BeBee México,Full-time,False,Mexico,2023-01-14 13:18:07,False,False,Mexico,None,NaN,NaN,Hewlett Packard Enterprise,"['r', 'python', 'sql', 'nosql', 'power bi', 't...","{'analyst_tools': ['power bi', 'tableau'], 'pr..."
2,Data Engineer,"Data Engineer/Scientist/Analyst, Mid or Senior...","Berlin, Germany",via LinkedIn,Full-time,False,Germany,2023-10-10 13:14:55,False,False,Germany,None,NaN,NaN,ALPHA Augmented Services,"['python', 'sql', 'c#', 'azure', 'airflow', 'd...","{'analyst_tools': ['dax'], 'cloud': ['azure'],..."
3,Data Engineer,LEAD ENGINEER - PRINCIPAL ANALYST - PRINCIPAL ...,"San Antonio, TX",via Diversity.com,Full-time,False,"Texas, United States",2023-07-04 13:01:41,True,False,United States,None,NaN,NaN,Southwest Research Institute,"['python', 'c++', 'java', 'matlab', 'aws', 'te...","{'cloud': ['aws'], 'libraries': ['tensorflow',..."
4,Data Engineer,Data Engineer- Sr Jobs,"Washington, DC",via Clearance Jobs,Full-time,False,Sudan,2023-08-07 14:29:36,False,False,Sudan,None,NaN,NaN,Kristina Daniel,"['bash', 'python', 'oracle', 'aws', 'ansible',...","{'cloud': ['oracle', 'aws'], 'other': ['ansibl..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785736,Software Engineer,DevOps Engineer,Singapura,melalui Trabajo.org,Pekerjaan tetap,False,Singapore,2023-03-13 06:16:16,False,False,Singapore,None,NaN,NaN,CAREERSTAR INTERNATIONAL PTE. LTD.,"['bash', 'python', 'perl', 'linux', 'unix', 'k...","{'os': ['linux', 'unix'], 'other': ['kubernete..."
785737,Data Analyst,CRM Data Analyst,"Bad Rodach, Jerman",melalui BeBee Deutschland,Pekerjaan tetap,False,Germany,2023-03-12 06:18:18,False,False,Germany,None,NaN,NaN,HABA FAMILYGROUP,"['sas', 'sas', 'sql', 'excel']","{'analyst_tools': ['sas', 'excel'], 'programmi..."
785738,Business Analyst,Commercial Analyst - Start Now,Malaysia,melalui Ricebowl,Pekerjaan tetap,False,Malaysia,2023-03-12 06:32:36,False,False,Malaysia,None,NaN,NaN,Lendlease Corporation,"['powerpoint', 'excel']","{'analyst_tools': ['powerpoint', 'excel']}"
785739,Data Engineer,"Principal Associate, Data Engineer (Remote-Eli...","Newark, New Jersey, Amerika Serikat",melalui Recruit.net,Pekerjaan tetap,False,Sudan,2023-03-12 06:32:15,False,False,Sudan,None,NaN,NaN,Capital One,"['python', 'go', 'nosql', 'sql', 'mongo', 'she...","{'cloud': ['aws', 'snowflake', 'azure', 'redsh..."


In [33]:
# let's calculate the median values in order to fill in for the missing values

median_salary_year = df["salary_year_avg"].median()

print(median_salary_year)

median_salary_hour = df["salary_hour_avg"].median()

print(median_salary_hour)

115000.0
45.97999954223633


In [34]:
# how do we fill in the empty values without replacing the ones with info?
# pandas has the pandas.DataFrame.fillna() method :)

# always create another df to keep the original one intact
df_filled = df

# we then update the info for each column
# we have to put it equal to itself to actually replace the NaN values
df_filled["salary_year_avg"] = df["salary_year_avg"].fillna(median_salary_year)
df_filled["salary_hour_avg"] = df["salary_hour_avg"].fillna(median_salary_hour)

# checking that it worked
df.loc[:, "salary_rate":"salary_hour_avg"]

,salary_rate,salary_year_avg,salary_hour_avg
0,None,115000.0,45.98
1,None,115000.0,45.98
2,None,115000.0,45.98
3,None,115000.0,45.98
4,None,115000.0,45.98
...,...,...,...
785736,None,115000.0,45.98
785737,None,115000.0,45.98
785738,None,115000.0,45.98
785739,None,115000.0,45.98


In [35]:
# now let's drop duplicates.
# pandas has a method: pandas.DataFrame.drop_duplicates() that returns a df with duplicate rows remove

# new df
df_unique = df_filled

# removing duplicate rows (based on all columns)
df_unique = df_unique.drop_duplicates()

# checking how many rows have been removed
print("Length of original df:           ", len(df_filled))
print("Length of drop duplicates df:    ", len(df_unique))
print("Rows dropped:                    ", len(df_filled)-len(df_unique))

Length of original df:            785741
Length of drop duplicates df:     785640
Rows dropped:                     101


In [36]:
# the above doesn't find many exact duplicates across all columns. 
# let's precise the columns we want to use to search for duplicates: job_title and company_name

# removing duplicate rows
df_unique = df_unique.drop_duplicates(subset=["job_title", "company_name"])

# checking how many rows have been removed
print("Length of original df:           ", len(df_filled))
print("Length of drop duplicates df:    ", len(df_unique))
print("Rows dropped:                    ", len(df_filled)-len(df_unique))

Length of original df:            785741
Length of drop duplicates df:     508042
Rows dropped:                     277699


# 2 Problems

## 2.2.1

Remove rows where the salary_year_avg column has missing values. Display the number of rows before and after removing.

In [46]:
df_cleaned = df

df_cleaned = df_cleaned.dropna(subset=["salary_year_avg"])

print("Length of original dataframe:    ", len(df))
print("Length of update dataframe:      ", len(df_cleaned))
print("Rows with missing salary info:   ", len(df)-len(df_cleaned))

# this is giving us the same number of rows because we have filled the NaN values above :)

Length of original dataframe:     785741
Length of update dataframe:       785741
Rows with missing salary info:    0


## 2.2.2

Remove duplicate rows from the DataFrame based on the job_location column. Display the number of rows before and after removing duplicates.

In [48]:
df_location_clean = df

df_location_clean = df_location_clean.drop_duplicates(subset=["job_location"])

print("Length of original dataframe:    ", len(df))
print("Length of update dataframe:      ", len(df_location_clean))
print("Rows with duplicate info:   ", len(df)-len(df_location_clean))

Length of original dataframe:     785741
Length of update dataframe:       17218
Rows with missing salary info:    768523


## 2.2.3

Fill missing values in the salary_rate column with the string 'Unknown'. Display the first 10 rows of the salary_rate column before and after filling.

In [51]:
df_salary_rate_filled = df

df_salary_rate_filled = df_salary_rate_filled.fillna({"salary_rate": "Unknown"})

print(df.loc[:10, "salary_rate"])
print(df_salary_rate_filled.loc[:10, "salary_rate"])

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
Name: salary_rate, dtype: object
0     Unknown
1     Unknown
2     Unknown
3     Unknown
4     Unknown
5     Unknown
6     Unknown
7     Unknown
8     Unknown
9     Unknown
10    Unknown
Name: salary_rate, dtype: object
